Global Score for Causal SHAP IBS

In [2]:
import pickle

# Define the path to the pickle file
file_path = '../../../result/phi_normalized_list.pkl'

# Load the data
with open(file_path, 'rb') as f:
    phi_normalized_list = pickle.load(f)

# Print or inspect the loaded data
print(phi_normalized_list)


[{'xylose': np.float64(0.13191601957276358), 'xanthosine': np.float64(-0.0012302754555192817), 'uracil': np.float64(-0.009612534286256316), 'ribulose/xylulose': np.float64(0.1398052719723667), 'valylglutamine': np.float64(-7.386755808860697e-05), 'tryptophylglycine': np.float64(0.0008794064947207714), 'succinate': np.float64(0.006962600356634677), 'valine betaine': np.float64(-0.0013035280526786234), 'ursodeoxycholate sulfate (1)': np.float64(-0.00032805778442085074), 'tricarballylate': np.float64(-0.00047464718617616045), 'succinimide': np.float64(-0.0004265953665802729), 'thymine': np.float64(0.007043544021562201), 'syringic acid': np.float64(0.0001624039492915659), 'serotonin': np.float64(-0.00980120401788335), 'ribitol': np.float64(-0.001182095205551476)}, {'xylose': np.float64(0.09599573609042962), 'xanthosine': np.float64(-0.0007661565610683617), 'uracil': np.float64(0.003386883640858132), 'ribulose/xylulose': np.float64(0.07726273085961621), 'valylglutamine': np.float64(2.663054

In [5]:
print(phi_normalized_list[0])

{'xylose': np.float64(0.13191601957276358), 'xanthosine': np.float64(-0.0012302754555192817), 'uracil': np.float64(-0.009612534286256316), 'ribulose/xylulose': np.float64(0.1398052719723667), 'valylglutamine': np.float64(-7.386755808860697e-05), 'tryptophylglycine': np.float64(0.0008794064947207714), 'succinate': np.float64(0.006962600356634677), 'valine betaine': np.float64(-0.0013035280526786234), 'ursodeoxycholate sulfate (1)': np.float64(-0.00032805778442085074), 'tricarballylate': np.float64(-0.00047464718617616045), 'succinimide': np.float64(-0.0004265953665802729), 'thymine': np.float64(0.007043544021562201), 'syringic acid': np.float64(0.0001624039492915659), 'serotonin': np.float64(-0.00980120401788335), 'ribitol': np.float64(-0.001182095205551476)}


In [35]:
sums = {}
counts = {}

for instance in phi_normalized_list:
    for metabolite, value in instance.items():
        sums[metabolite] = sums.get(metabolite, 0) + abs(value)
        counts[metabolite] = counts.get(metabolite, 0) + 1

means = {metabolite: sums[metabolite] / counts[metabolite] for metabolite in sums}

sorted_means = sorted(means.items(), key=lambda item: abs(item[1]), reverse=True)

causal_output_list = [(metabolite, mean_value) for metabolite, mean_value in sorted_means]

print(causal_output_list)

[('xylose', np.float64(0.14249255678910933)), ('ribulose/xylulose', np.float64(0.1324045675579535)), ('serotonin', np.float64(0.08015593469032148)), ('valine betaine', np.float64(0.024109210282489253)), ('thymine', np.float64(0.01845642393468088)), ('succinate', np.float64(0.01434662259345521)), ('ribitol', np.float64(0.01244294416240047)), ('uracil', np.float64(0.010309077873849807)), ('succinimide', np.float64(0.004449842958519002)), ('xanthosine', np.float64(0.0037146766543883904)), ('tryptophylglycine', np.float64(0.0012059463203196253)), ('valylglutamine', np.float64(0.0007212578112272749)), ('tricarballylate', np.float64(0.0006481119920928386)), ('syringic acid', np.float64(0.0005111378807435147)), ('ursodeoxycholate sulfate (1)', np.float64(0.0003253555581938992))]


Original SHAP

In [15]:
import argparse
from pathlib import Path
from data_processing import DataProcessor
from models import ModelTrainer
from feature_selection import FeatureSelector
from visualization import Visualizer
from causal_inference import CausalInference
import shap
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import matplotlib
from evaluation import iterative_feature_deletion_with_rmse, iterative_feature_addition_with_rmse

import warnings
warnings.filterwarnings("ignore")



c:\Users\snorl\Desktop\FYP\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
DEBUG:matplotlib:matplotlib data path: c:\Users\snorl\Desktop\FYP\venv\Lib\site-packages\matplotlib\mpl-data
DEBUG:matplotlib:CONFIGDIR=C:\Users\snorl\.matplotlib
DEBUG:matplotlib:interactive is False
DEBUG:matplotlib:platform is win32
DEBUG:matplotlib:CACHEDIR=C:\Users\snorl\.matplotlib
DEBUG:matplotlib.font_manager:Using fontManager instance from C:\Users\snorl\.matplotlib\fontlist-v390.json
DEBUG:pydot:pydot initializing
DEBUG:pydot:pydot 3.0.2
DEBUG:pydot.dot_parser:pydot dot_parser module initializing
DEBUG:pydot.core:pydot core module initializing


In [18]:
print("Starting ML Pipeline...")
base_dir = '../../../'
print(f"Base directory set to: {base_dir}")

data_path = base_dir + 'dataset/' + 'data_full.xlsx'
raw_data_path = base_dir + 'dataset/' + 'result_raw.xlsx'
result_dir = base_dir + 'result/R/'

report_file_path = result_dir + 'report.txt'

print("Loading data...")
data_processor = DataProcessor(data_path=str(data_path))
df = data_processor.load_data_metabolites()
print("Data loaded successfully.")

print("Encoding labels...")
df_encoded, label_encoder = data_processor.encode_labels(df, label_column='Group')
print("Labels encoded successfully.")
X = df_encoded.drop(columns=['Group'])
y = df_encoded['Group']

Starting ML Pipeline...
Base directory set to: ../../../
Loading data...
Data loaded successfully.
Encoding labels...
Labels encoded successfully.


In [19]:
X = X[["xylose", "xanthosine", "uracil", "ribulose/xylulose", "valylglutamine", "tryptophylglycine", "succinate", "valine betaine", "ursodeoxycholate sulfate (1)", "tricarballylate","succinimide", "thymine", "syringic acid", "serotonin", "ribitol" ]]

y = df_encoded['Group']

print("Training Random Forest model...")
param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 7],
    'min_samples_leaf': [1, 2, 4]
}
model_trainer = ModelTrainer(X, y)
model, best_params = model_trainer.train_random_forest(param_dist)

Training Random Forest model...
Fitting 3 folds for each of 50 candidates, totalling 150 fits


In [34]:
print("Applying SHAP for explainability...")

explainer = shap.TreeExplainer(model, model_trainer.X_train)

shap_values = explainer.shap_values(model_trainer.X_test)

feature_names = model_trainer.X_test.columns

mean_abs_shap_values = np.mean(np.abs(shap_values[1]), axis=1)

shap_importance = list(zip(feature_names, mean_abs_shap_values))

shap_importance_sorted = sorted(shap_importance, key=lambda x: x[1], reverse=True)

shap_output_list = [(feature, importance) for feature, importance in shap_importance_sorted]

print(shap_output_list)
    


Applying SHAP for explainability...
[('xylose', np.float64(0.10000743516657166)), ('serotonin', np.float64(0.060477922572854365)), ('ribulose/xylulose', np.float64(0.05253665230959996)), ('ribitol', np.float64(0.04543462194515541)), ('valine betaine', np.float64(0.016420123335110476)), ('thymine', np.float64(0.010505644627664878)), ('tryptophylglycine', np.float64(0.007556361510496572)), ('syringic acid', np.float64(0.004192226959253276)), ('tricarballylate', np.float64(0.0033999778656198035)), ('uracil', np.float64(0.0023097257251424707)), ('succinimide', np.float64(0.0022228713183267248)), ('xanthosine', np.float64(0.002033165061445743)), ('succinate', np.float64(0.0018616566714058536)), ('ursodeoxycholate sulfate (1)', np.float64(0.0006252102878079313)), ('valylglutamine', np.float64(0.0001221017460420626))]


In [ ]:
print(causal_output_list)
print(shap_output_list)

[('xylose', np.float64(0.14249255678910933)), ('ribulose/xylulose', np.float64(0.1324045675579535)), ('serotonin', np.float64(0.08015593469032148)), ('valine betaine', np.float64(0.024109210282489253)), ('thymine', np.float64(0.01845642393468088)), ('succinate', np.float64(0.01434662259345521)), ('ribitol', np.float64(0.01244294416240047)), ('uracil', np.float64(0.010309077873849807)), ('succinimide', np.float64(0.004449842958519002)), ('xanthosine', np.float64(0.0037146766543883904)), ('tryptophylglycine', np.float64(0.0012059463203196253)), ('valylglutamine', np.float64(0.0007212578112272749)), ('tricarballylate', np.float64(0.0006481119920928386)), ('syringic acid', np.float64(0.0005111378807435147)), ('ursodeoxycholate sulfate (1)', np.float64(0.0003253555581938992))]
[('xylose', np.float64(0.10000743516657166)), ('serotonin', np.float64(0.060477922572854365)), ('ribulose/xylulose', np.float64(0.05253665230959996)), ('ribitol', np.float64(0.04543462194515541)), ('valine betaine', n

In [38]:
import numpy as np
import pandas as pd
from sklearn.metrics import mean_squared_error

def iterative_feature_deletion_with_rmse_over_testset(model, X_test, attribution_scores, top_k=None):
    """
    Iteratively deletes features based on attribution scores and computes the average RMSE over all instances in the test set.

    Parameters:
    - model: Trained Scikit-learn classifier.
    - X_test (pd.DataFrame): Test set features.
    - attribution_scores (dict): Feature attributions (e.g., SHAP values).
    - top_k (int): Number of top features to delete. If None, deletes all features.

    Returns:
    - avg_rmse (float): Average RMSE after deletions over the test set.
    """
    if top_k is None:
        top_k = X_test.shape[1]

    # Ensure attribution_scores is a dict
    if not isinstance(attribution_scores, dict):
        raise TypeError("Attribution scores must be a dictionary with feature names as keys and scores as values.")

    # Sort features by absolute attribution scores in descending order
    sorted_features = sorted(X_test.columns, key=lambda x: -abs(attribution_scores.get(x, 0)))

    # List to store RMSE values for each instance
    rmse_values = []

    # Iterate over each instance in the test set
    for index, input_features in X_test.iterrows():
        # Original prediction probability for class 1
        y_predicted = model.predict_proba([input_features.values])[0][1]

        # Copy the input features to modify
        modified_input = input_features.copy().astype(float)

        # List to store RMSE values at each step for this instance
        instance_rmse_values = []

        # Iteratively delete features and compute RMSE
        for i in range(top_k):
            feature_to_delete = sorted_features[i]
            modified_input[feature_to_delete] = 0.0  # Set feature to zero
            prediction = model.predict_proba([modified_input.values])[0][1]
            rmse = np.sqrt(mean_squared_error([y_predicted], [prediction]))
            instance_rmse_values.append(rmse)

        # Average RMSE for this instance
        instance_avg_rmse = np.mean(instance_rmse_values)
        rmse_values.append(instance_avg_rmse)

    # Compute overall average RMSE across all instances
    overall_avg_rmse = np.mean(rmse_values)
    return overall_avg_rmse


def iterative_feature_addition_with_rmse_over_testset(model, X_test, attribution_scores, top_k=None):
    """
    Iteratively adds features based on attribution scores and computes the average RMSE over all instances in the test set.

    Parameters:
    - model: Trained Scikit-learn classifier.
    - X_test (pd.DataFrame): Test set features.
    - attribution_scores (dict): Feature attributions (e.g., SHAP values).
    - top_k (int): Number of top features to add. If None, adds all features.

    Returns:
    - avg_rmse (float): Average RMSE after additions over the test set.
    """
    if top_k is None:
        top_k = X_test.shape[1]

    # Ensure attribution_scores is a dict
    if not isinstance(attribution_scores, dict):
        raise TypeError("Attribution scores must be a dictionary with feature names as keys and scores as values.")

    # Sort features by absolute attribution scores in descending order
    sorted_features = sorted(X_test.columns, key=lambda x: -abs(attribution_scores.get(x, 0)))

    # List to store RMSE values for each instance
    rmse_values = []

    # Iterate over each instance in the test set
    for index, input_features in X_test.iterrows():
        # Original prediction probability for class 1
        y_predicted = model.predict_proba([input_features.values])[0][1]

        # Initialize modified input with zeros
        modified_input = pd.Series(0.0, index=input_features.index)

        # List to store RMSE values at each step for this instance
        instance_rmse_values = []

        # Iteratively add features and compute RMSE
        for i in range(top_k):
            feature_to_add = sorted_features[i]
            modified_input[feature_to_add] = input_features[feature_to_add].astype(float)
            prediction = model.predict_proba([modified_input.values])[0][1]
            rmse = np.sqrt(mean_squared_error([y_predicted], [prediction]))
            instance_rmse_values.append(rmse)

        # Average RMSE for this instance
        instance_avg_rmse = np.mean(instance_rmse_values)
        rmse_values.append(instance_avg_rmse)

    # Compute overall average RMSE across all instances
    overall_avg_rmse = np.mean(rmse_values)
    return overall_avg_rmse



In [39]:
# Convert causal_output_list to a dictionary
causal_attribution_scores = dict(causal_output_list)

# Convert shap_output_list to a dictionary
shap_attribution_scores = dict(shap_output_list)


In [ ]:
# Compute RMSE over the test set using causal attributions
causal_rmse_deletion = iterative_feature_deletion_with_rmse_over_testset(
    model, model_trainer.X_test, causal_attribution_scores)

causal_rmse_addition = iterative_feature_addition_with_rmse_over_testset(
    model, model_trainer.X_test, causal_attribution_scores)

# Compute RMSE over the test set using SHAP attributions
shap_rmse_deletion = iterative_feature_deletion_with_rmse_over_testset(
    model, model_trainer.X_test, shap_attribution_scores)

shap_rmse_addition = iterative_feature_addition_with_rmse_over_testset(
    model, model_trainer.X_test, shap_attribution_scores)



In [42]:
print("Using Global Score to sort Features")
print("SHAP RMSE after feature deletion over test set:", shap_rmse_deletion)
print("Causal RMSE after feature deletion over test set:", causal_rmse_deletion)
print("SHAP RMSE after feature addition over test set:", shap_rmse_addition)
print("Causal RMSE after feature addition over test set:", causal_rmse_addition)

Using Global Score to sort Features
SHAP RMSE after feature deletion over test set: 0.19212695621359016
Causal RMSE after feature deletion over test set: 0.19743585361592064
SHAP RMSE after feature addition over test set: 0.09998943786572828
Causal RMSE after feature addition over test set: 0.0874046547044305
